# Lesson 2: Guidelines for Prompting
https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines

In this lesson, we'll practice two prompting principles and their related tactics in order to write effective prompts for LLM.

In [1]:
import os

import openai
from dotenv import find_dotenv, load_dotenv

if not load_dotenv(find_dotenv()):
    raise RuntimeError("Set environment variables first!")

if os.getenv("OPENAI_API_KEY"):
    print("API key found")
else:
    raise RuntimeError("No API key found!")

client = openai.Client()

API key found


In [2]:
MODEL_NAME = "gpt-3.5-turbo-0125"

In [3]:
def get_completion(prompt: str, model: str = MODEL_NAME) -> None:
    """Helper function to make it easier to use prompts and look at the
    generated outpus"""

    messages = [{"role": "user", "name": "bob", "content": prompt}]

    stream = client.chat.completions.create(
        messages=messages, model=model, stream=True, temperature=0.0
    )

    for part in stream:
        print(part.choices[0].delta.content or "", end="")

Prompting Principles

1. Write clear and specific instructions
2. Give the model time to "think"


## Principle 1: Write clear and specific instructions

### Tactics 1: Use delimiters to clearly indicate distinct parts of the input.

In [4]:
text = """
You should express what you want a model to do by \
providing instructions that are as clear and \
specific as you can possibly make them. \
This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant \
or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. \
In many cases, longer prompts provide more clarity \
and context for the model, which can lead to \
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks \
into a single sentence.
```{text}```
"""

get_completion(prompt=prompt)

Providing clear and specific instructions to a model is essential for guiding it towards the desired output and reducing the chances of receiving irrelevant or incorrect responses, even if longer prompts are necessary for clarity and context.

### Tactic 2: Ask for a structured output.

In [5]:
prompt = """
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""

get_completion(prompt=prompt)

[
    {
        "book_id": 1,
        "title": "The Midnight Garden",
        "author": "Eleanor Blackwood",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "Echoes of the Past",
        "author": "Julian Rivers",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "Whispers in the Wind",
        "author": "Serena Nightingale",
        "genre": "Romance"
    }
]

### Tactic 3: Ask the model to check whether conditions are satisfied.

In [25]:
text_1 = """
Making a cup of tea is easy! First, you need to get some \
water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is \
hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a \
few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. \
And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

print("Completion for Text 1:")
get_completion(prompt=prompt)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Remove the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.None


In [26]:
text_2 = """
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""

prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""

print("Completion for Text 2:")
get_completion(prompt=prompt)

Completion for Text 2:
No steps provided.None


### Tactic 4: "Few-shot" prompting

In [27]:
prompt = """
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \
valley flows from a modest spring; the \
grandest symphony originates from a single note; \
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

get_completion(prompt=prompt)

<grandparent>: Resilience is like the mighty oak tree that withstands the fiercest storms, bending but never breaking. It is the ability to bounce back from adversity, to find strength in the face of challenges, and to keep moving forward despite setbacks. Just like the oak tree, resilience grows stronger with each trial it endures.None


## Principle 2: Give the model time to “think”

### Tactic 1: Specify the steps required to complete a task

In [28]:
text = """
In a charming village, siblings Jack and Jill set out on \
a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune \
struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. \
Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, \
their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""

# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

print("Completion for prompt 1:")
get_completion(prompt=prompt_1)

Completion for prompt 1:
1 - Jack and Jill go on a quest to fetch water, but Jack trips and tumbles down a hill, with Jill following, yet they return home with comforting embraces and continue exploring with delight.

2 - Jack et Jill partent en quête d'eau, mais Jack trébuche et dégringole d'une colline, avec Jill qui suit, mais ils rentrent chez eux avec des étreintes réconfortantes et continuent à explorer avec délice.

3 - Jack, Jill

4 - 
{
  "french_summary": "Jack et Jill partent en quête d'eau, mais Jack trébuche et dégringole d'une colline, avec Jill qui suit, mais ils rentrent chez eux avec des étreintes réconfortantes et continuent à explorer avec délice.",
  "num_names": 2
}

Ask for output in a specified format

In [30]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""

print("Completion for prompt 2:")
get_completion(prompt=prompt_2)

Completion for prompt 2:
Summary: Jack and Jill, siblings, go on a quest to fetch water but encounter misfortune on the way back.
Translation: Jack et Jill, frère et sœur, partent en quête d'eau mais rencontrent des malheurs sur le chemin du retour.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill, frère et sœur, partent en quête d'eau mais rencontrent des malheurs sur le chemin du retour.", "num_names": 2}

### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion.

In [31]:
prompt = """
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

get_completion(prompt=prompt)

The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is indeed 450x + 100,000.None


Note that the student's solution is actually not correct.
We can fix this by instructing the model to work out its own solution first.

In [32]:
prompt = """
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution \
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""

get_completion(prompt=prompt)

To calculate the total cost for the first year of operations as a function of the number of square feet, we need to add up the land cost, solar panel cost, and maintenance cost.

Let x be the size of the installation in square feet.
Costs:
1. Land cost: $100 * x
2. Solar panel cost: $250 * x
3. Maintenance cost: $100,000 + $10 * x

Total cost: $100x + $250x + $100,000 + $10x = $360x + $100,000

Is the student's solution the same as actual solution just calculated:
```
Yes
```
Student grade:
```
correct
```None


## Model Limitations: Hallucinations

In [33]:
# Boie is a real company, the product name is not real.
prompt = """Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie"""

get_completion(prompt=prompt)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush designed to provide a superior cleaning experience. It features an ultra-slim design that allows for easy maneuverability and access to hard-to-reach areas of the mouth. The toothbrush also includes smart technology that tracks brushing habits and provides real-time feedback to help users improve their oral hygiene routine.

The brush head is made from durable, long-lasting materials that are gentle on the teeth and gums, and the bristles are designed to effectively remove plaque and debris from the teeth. The toothbrush also features a long-lasting battery that can be easily recharged, making it convenient for travel and everyday use.

Overall, the AeroGlide UltraSlim Smart Toothbrush by Boie is a sleek and innovative toothbrush that offers advanced cleaning technology and a comfortable, user-friendly design.